In [ ]:
### import dependencies 

import pandas as pd

from __future__ import print_function

import os.path

import pygsheets

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [ ]:
### generate dictionary of parent folders

#for this example, there is only one folder in the list
folderList = {'FOLDER NAME' : 'FOLDER ID'}

In [ ]:
### variation on code to return list of files

# define scope - by convention this variable is SCOPES
SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']

# function to return list of files 
# paramater is length of list
def getFileList():

    creds = None
    # The file token.json stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials2.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Connect to the API service
    try:
        service = build('drive', 'v3', credentials=creds)

    except HttpError as error:
        # TODO(developer) - Handle errors from drive API.
        print(f'An error occurred: {error}')
  
    page_token = None
    dict = {}
    listCount = 0
    while True:
        last = list(folderList.values())[listCount]
        query = "'" + last + "' in parents"
        print(listCount, last)
        response = service.files().list(q=query,
                                            spaces='drive',
                                            fields='nextPageToken, files(name, id, parents)',
                                            pageToken=page_token).execute()
        for file in response.get('files', []):
            # Process change
            dict.update({file.get('name'):[file.get('id'),file.get('parents')]})
        page_token = response.get('nextPageToken', None)
        listCount = listCount + 1
        if listCount == len(folderList):
            return dict

df = pd.DataFrame.from_dict(getFileList(),orient='index',columns=['id','parent'])

In [ ]:
### set up DataFrame of files, ids, and parents

df1 = df
df1['parent'] = df1['parent'].str.join(', ')
df2 = df1.reset_index()
df2.head()#.iloc[:,0][0]

In [ ]:
### function to get the name of all sheets in a file

def getAllSheets(dfOfFiles,iCount):
    gc = pygsheets.authorize(service_file='pygsheets.json')
    # open google spreadsheet
    sh = gc.open(dfOfFiles.iloc[0:,0][iCount])
    # define which sheet to open by sheet name
    wks = sh.worksheets()
    return(wks)

In [ ]:
### funtion to transform list of pygsheets worksheets into a list of strings

def dfOfWks(wks):
    itemCount = 0
    listOf = []
    #df = pd.DataFrame()
    while True:
        sheetName = wks[itemCount].title
        listOf.append(sheetName)
        itemCount += 1
        if itemCount == len(wks):
            return(listOf)

In [ ]:
### function to generate DataFrame with all sheets in a given file

def dfOfWkbks(wBook,wSheet,iCount):
    dfWBook = pd.DataFrame(columns=['name','id','parent','sheet_name'])
    dfWBook = dfWBook.assign(file_name=lambda x: wSheet)
    dfWBook = dfWBook.assign(name=lambda x: wBook['index'].iloc[iCount])
    dfWBook = dfWBook.assign(id=lambda x: wBook['id'].iloc[iCount])
    dfWBook = dfWBook.assign(parent=lambda x: wBook['parent'].iloc[iCount])
    return(dfWBook)        

In [ ]:
### function to getnerate DataFrame with all sheets in all files

def makeList(dfOfFiles):
    iCount = 0
    df40 = pd.DataFrame()
    while True:
        df10 = getAllSheets(dfOfFiles,iCount)
        df20 = dfOfWks(df10)
        df30 = dfOfWkbks(dfOfFiles,df20,iCount)
        df40 = pd.concat([df40,df30])
        iCount += 1
        if iCount == len(dfOfFiles):
            return(df40)

dfFinal = makeList(df2)
dfFinal.shape


In [ ]:
### output file name list for use in next stage

dfFinal.to_csv('files_list.csv')